# Plotting the maximum radius and corresponding P_ZAMS

When computing the formation efficiency, one factor that comes into play is the range of periods of stars that interact

To estimate this, I want to plot the maximum radius of stars at different metallicities for a typical BH and NS progenitor star, and then plot the corresponging P_ZAMS 

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import h5py as h5
from matplotlib.colors import ListedColormap
from astropy.table import Table

from astropy import constants as const
from astropy import units as u

home_dir = os.path.expanduser("~")

######################################
## PLOT setttings
plt.rc('font', family='serif')
from matplotlib import rc
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
rc('text', usetex=False)
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,18,25,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=MEDIUM_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize


######################################
sim_name        = "OldWinds_RemFryer2012" # sim_name determines which flags to run COMPAS with


: 

# Load the Single star grid data

In [ ]:
# Mapping types to names 
Stellar_Type_table = Table()
Stellar_Type_table['types'] = ['MS_low', 'MS' ,'HG', 'AGB', 'CHeB', 'EAGB','TPAGB','HeMS','HeHG','HeGB','HeWD','COWD','ONeWD','NS','BH','massless remnant']
Stellar_Type_table['indices'] = np.arange(len(Stellar_Type_table))

# NEW mass loss types
Wind_prescr_table = Table()
Wind_prescr_table['types'] = ['NONE', 'OB', 'VMS', 'GB', 'RSG', 'WR', 'LBV']
Wind_prescr_table['indices'] = np.arange(len(Wind_prescr_table))

# # Old mass loss types
# Wind_prescr_table = Table()
# Wind_prescr_table['types'] = ['None', 'Nieuwenhuijzen and de Jager', 'Kudritzki and Reimers', 'Vassiliadis and Wood', 'Wolf-Rayet-like (Hamann, Koesterke and de Koter)', 'Vink', 'Luminous Blue Variable']
# Wind_prescr_table['indices'] = np.arange(len(Wind_prescr_table))


def get_max_radii(loc = '{root_out_dir}/SingleStarGrids/SingleStars_M10/'):
    Metals, maxRadii, maxR_type, maxR_massloss    = [],  [],  [],  []

    for i in range(100):
        # Read detailed hdf5 file
        detailed_data = f'{loc}/Detailed_Output/BSE_Detailed_Output_{i}.h5'
        detailh5 = h5.File(detailed_data, 'r')

        # Get the relevant info out
        i_maxR = np.argmax(detailh5['Radius(1)'][()])
        Metals.append(detailh5['Metallicity@ZAMS(1)'][0])
        maxRadii.append( detailh5['Radius(1)'][i_maxR] )
        maxR_type.append(detailh5['Stellar_Type(1)'][i_maxR])   
        maxR_massloss.append(detailh5['Dominant_Mass_Loss_Rate(1)'][i_maxR])

        maxR_RL1 = np.max(detailh5['Radius(1)'][()]/detailh5['RocheLobe(1)'][()]) 
        if maxR_RL1 > 1.0:
            print(f'Watch out!RL1: {maxR_RL1} for {i}')
        maxR_RL2 = np.max(detailh5['Radius(2)'][()]/detailh5['RocheLobe(2)'][()]) 
        if maxR_RL2 > 1.0:
            print(f'Watch out! RL2: {maxR_RL2} for {i}')
    return Metals, maxRadii, maxR_type, maxR_massloss

In [ ]:
# Initialize an empty dictionary
data = {
    'M0': [],
    'Metals': [],
    'maxRadii': [],
    'maxR_type': [],
    'maxR_massloss': [],
    #
    'R_ZAMS': [],
    'maxR_MS': [],
    'maxR_HG': [],
    'maxR_FGB': [],
    'maxR_CHeB': [],
    'maxR_EAGB': [],
    #
    'Luminosity': [],
    'Teff': [],
    'Radius': [],
    'Mass': [],
    'Stellar_Type': [],
    'Dominant_Wind': []
}

# For each mass, compute the values and append them to the lists
for mass in [10, 20, 30, 40, 50, 60, 70,90, 100, 150]:
    print('mass', mass)

    read_loc =  f'{root_out_dir}/SingleStarGrids/SingleStars_M{mass}/'

    print('reading ', read_loc)
    
    try:
        Metals, maxRadii, maxR_type, maxR_massloss   = [],  [],  [],  []
        R_ZAMS, maxR_MS, maxR_HG, maxR_FGB, maxR_CHeB, maxR_EAGB = [], [], [], [], [], []
        L, Teff, M, Radius, stellar_type, dominant_wind = [], [], [], [], [], []

        for i in range(100):
            # Read detailed hdf5 file
            detailh5 = h5.File(f'{read_loc}/Detailed_Output/BSE_Detailed_Output_{i}.h5', 'r')

            ####################################### 
            # Maximum radius at all times
            i_maxR = np.argmax(detailh5['Radius(1)'][()])
            Metals.append(detailh5['Metallicity@ZAMS(1)'][0])
            maxRadii.append( detailh5['Radius(1)'][i_maxR] )
            maxR_type.append(detailh5['Stellar_Type(1)'][i_maxR])   
            maxR_massloss.append(detailh5['Dominant_Mass_Loss_Rate(1)'][i_maxR])

            #######################################
            # Select the maximum radius for each stellar type
            R_ZAMS.append(detailh5['Radius(1)'][0])

            st1 = detailh5['Stellar_Type(1)'][()]
            MS_bool = np.logical_or(st1 == 0, st1 == 1)
            maxR_MS.append(max(detailh5['Radius(1)'][MS_bool]) )

            HG_bool = st1 == 2
            maxR_HG.append(max(detailh5['Radius(1)'][HG_bool]) if np.any(HG_bool) > 0 else 0)

            FGB_bool = st1 == 3
            maxR_FGB.append(max(detailh5['Radius(1)'][FGB_bool]) if np.any(FGB_bool) > 0 else 0)
            
            CHeB_bool = st1 == 4
            maxR_CHeB.append(max(detailh5['Radius(1)'][CHeB_bool]) if np.any(CHeB_bool) > 0 else 0 )

            EAGB_bool = st1 == 5
            maxR_EAGB.append(max(detailh5['Radius(1)'][EAGB_bool]) if np.any(EAGB_bool) > 0 else 0 )

            #######################################
            # For HR diagram
            L.append(detailh5['Luminosity(1)'])
            Teff.append(detailh5['Teff(1)'])
            Radius.append(detailh5['Radius(1)'])
            M.append(detailh5['Mass(1)'])
            stellar_type.append(detailh5['Stellar_Type(1)'])
            dominant_wind.append(detailh5['Dominant_Mass_Loss_Rate(1)'])

            maxR_RL1 = np.max(detailh5['Radius(1)'][()]/detailh5['RocheLobe(1)'][()]) 
            if maxR_RL1 > 1.0:
                print(f'Watch out! maxR_RL1: {maxR_RL1} for {i}')
            maxR_RL2 = np.max(detailh5['Radius(2)'][()]/detailh5['RocheLobe(2)'][()]) 
            if maxR_RL2 > 1.0:
                print(f'Watch out! maxR_RL2: {maxR_RL2} for {i}')
                    
        # Max R
        data['M0'].append(mass)
        data['Metals'].append(Metals)
        data['maxRadii'].append(maxRadii)
        data['maxR_type'].append(maxR_type)
        data['maxR_massloss'].append(maxR_massloss)

        # Max R per type
        data['R_ZAMS'].append(R_ZAMS)
        data['maxR_MS'].append(maxR_MS)
        data['maxR_HG'].append(maxR_HG)
        data['maxR_FGB'].append(maxR_FGB)
        data['maxR_CHeB'].append(maxR_CHeB)
        data['maxR_EAGB'].append(maxR_EAGB)

        # HR
        data['Luminosity'].append(L)
        data['Teff'].append(Teff)   
        data['Radius'].append(Radius)  
        data['Mass'].append(M)  
        data['Stellar_Type'].append(stellar_type)  
        data['Dominant_Wind'].append(dominant_wind)  

    except Exception as e:
        print('ERR:', e)
        traceback.print_exc()
        continue


print(data['M0'])

mass 10
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M10/
mass 20
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M20/
mass 30
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M30/
mass 40
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M40/
mass 50
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M50/
mass 60
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M60/
mass 70
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M70/
mass 90
reading  /mnt/home/lvanson/ceph/CompasOutput/v02.46.01/NewWinds_RemMullerMandel/SingleStarGrids/SingleStars_M90/
mass 100
reading  /mnt/home/lvan